In [ ]:
import pandas as pd
import numpy as np
import datetime

# Procedura logowania
Dane użytkowików znajdują się w dataframie "users", który tworzony jest w kodzie poniżej. Procedura logowania składa się z zapytań do użytkownika o podanie emaila oraz hasła.

In [ ]:
#dane z użytkownikami
users = pd.DataFrame({'login': ['admin@bank.pl', 'user@bank.pl'], 'password': ['pass', 'word']})
login_history = pd.DataFrame(columns= ['email', 'password_correct','email_correct', 'datetime'])

#procedura logowania
def login_proc():
  email = input("Podaj adres email: ")
  if email not in users['login'].to_list():
    login_history.loc[len(login_history)] = [email, None, 'No', datetime.datetime.now().strftime('%Y-%m-%d')]
    return 'Błędna nazwa użytkowanika'
  else:
    passw = input("Podaj hasło: ")
    if passw == users[users['login'] == email]['password'].values[0]:
      login_history.loc[len(login_history)] = [email, 'Yes', 'Yes', datetime.datetime.now().strftime('%Y-%m-%d')]
      return "Zalogowano!"
    else:
      login_history.loc[len(login_history)] = [email, 'No', 'Yes', datetime.datetime.now().strftime('%Y-%m-%d')]
      return "Nieprawidłowe hasło"

users


,login,password
0,admin@bank.pl,pass
1,user@bank.pl,word


In [ ]:
#kolumny do modelu - pomijamy zmienne, których znaczenia nie udało nam się ustalić
cols = ['age','job','marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'previous', 'poutcome', 'deposit']


# Przygotowanie danych
Dane pobieramy z pliku csv. Plik z danymi pochodzi z platformy Kaggle.

In [ ]:
df = pd.read_csv("bank.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


Dane do kolumny wyjściowej zamieniamy na wartości 0 przy braku złożenia lokaty i 1 przy założeniu lokaty przez klienta.

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
df['deposit']=LE.fit_transform(df.deposit.values)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,1
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,1
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,1
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,1
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,1


Przed modelowaniem wydzielimy próbki do nauki i testu modelu. Zmienną, którą będziemy prognozować jest 'deposit'

In [ ]:
y=np.ravel(df['deposit'])
X = df[[item for item in cols if item not in ['deposit']]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

y_train

array([0, 1, 1, ..., 0, 1, 0])

Zmienne objaśnijące, które nie są zmiennymi ciągłymi, przekształcimy w zmienne binarne, rozszerzając macierz X o dodatkowe kolumny.

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


X_temp = X_train
X_train.head()

,age,balance,day,duration,campaign,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
7968,57,950,9,1272,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1456,34,925,13,709,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
361,41,75,1,479,9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
812,46,4414,18,504,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1092,28,4465,28,769,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


Zmienne ciągłe przenisemy do postaci znormalizowanej, czyli przenisemy wartości do jednej skali - od 0 do 1. Wykorzystamy do tego funkcję MinMaxScaler. Skalę oprzemy o próbkę testową, i to względem niej przkształcimy dane. Zmienne czynnikowe przekształciliśmy to zmiennych binarnych, więc przeskalowanie całych macierzy nie będzie problemem. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

X_test[3]

array([0.37662338, 0.13395125, 0.23333333, 0.20159835, 0.01612903,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 1.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ])

# Przygotowanie modelu
Do rozwiązania problemu użyjemy modelu Sequential z bilbioteki Keras.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(50, activation='relu', input_shape=(50,)))

model.add(Dense(46, activation='relu'))

model.add(Dense(20, activation='relu'))

model.add(Dense(22, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

Jako, że mamy problem klasyfikacji do dwóch klas, jako funkcji straty zastosujemy binary_crossentropy.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs = 4, batch_size=1, verbose=1)

Epoch 1/4
7813/7813 [==============================] - 9s 1ms/step - loss: 0.6372 - accuracy: 0.6190
Epoch 2/4
7813/7813 [==============================] - 8s 1ms/step - loss: 0.4989 - accuracy: 0.7670
Epoch 3/4
7813/7813 [==============================] - 8s 1ms/step - loss: 0.4503 - accuracy: 0.7956
Epoch 4/4
7813/7813 [==============================] - 8s 1ms/step - loss: 0.4205 - accuracy: 0.8154


Sprawdźmy skuteczność modelu

In [ ]:
y_pred = model.predict_classes(X_test)

score = model.evaluate(X_test, y_test,verbose=1)

print(score)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


105/105 [==============================] - 0s 1ms/step - loss: 0.3954 - accuracy: 0.8244
[0.3953533470630646, 0.824425220489502]


Zapisujemy model

In [ ]:
model.summary()
model.save('saved_model/term_deposit_classification_model')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 46)                2346      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                940       
_________________________________________________________________
dense_3 (Dense)              (None, 22)                462       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 23        
Total params: 6,321
Trainable params: 6,321
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: saved_model/term_deposit_classification_model/assets


# Procedura przyjmowania danych od użytkownika
Każda dana od użytkownika będzie przyjmowana od użytkownika za pomocą odzielnych funkcji:

In [ ]:

def ask_age():
  while True:
     try:
         u_age = int(input("Wprowadź wiek klienta: "))
     except ValueError:
         print("Wiek powinien być liczbą - przykładowo 24")
         continue
     else:
         break
  if u_age >= 18: 
     return u_age
  else:
     print("Klient musi być pełnoletni.")
     ask_age()

def ask_job():
  u_job = input("Podaj zawód klienta (przykładowo: admin., management,student, retired): ")
  if u_job not in ['admin.','technician','services','management','retired','blue-collar','unemployed','entrepreneur','housemaid','self-employed','student']:
    print("Zawód nierozpoznany - wprowadzono wartość 'unknown'")
    u_job = 'unknown'
  return u_job
  
def ask_marital():
  while True:
    u_mar = input('Wprowadź stan cywilny klienta. Możliwe opcje - married, single lub divorced: ')
    if u_mar in ['married','single','divorced']:
      return u_mar
    else:
      print('Wybierz opcję z listy - married, sigle lub divorced')

def ask_education():
  u_edu = input('Wprowadź informację na temat wykształcenia klienta. Możliwe opcje - primary, secondary, tertiary: ')
  if u_edu not in ['primary', 'lower secondary', 'secondary', 'tertiary']:
    print('Wykształcenie nierozpoznane - wprowadzono "unknown"')
    u_edu = 'unknown'
  return u_edu

def ask_default():
  while True:
    u_default = input('Czy klient ma status default? Y/N: ')
    if u_default.upper() in ['Y', 'N']:
      if u_default.upper() == 'Y': return 'yes'
      else: return 'no'
    else:
      print('Wybierz Y lub N')

def ask_balance():
  while True:
     try:
         u_bal = float(input("Wprowadź stan konta klienta: "))
     except ValueError:
         print("Wprowoadz kwotę")
         continue
     else:
         return u_bal

def ask_credit(i):
  if i == 1:
    while True:
     u_housing = input('Czy klient ma kredyt hipoteczny? Y/N: ')
     if u_housing.upper() in ['Y', 'N']:
       if u_housing.upper() == 'Y': return 'yes'
       else: return 'no'
     else:
       print('Wybierz Y lub N')
  else:
    while True:
     u_credit = input('Czy klient ma kredyt konsumencki? Y/N: ')
     if u_credit.upper() in ['Y', 'N']:
       if u_credit.upper() == 'Y': return 'yes'
       else: return 'no'
     else:
       print('Wybierz Y lub N')

def ask_contact():
  u_contact = input('Jaka jest preferowana forma kontaktu klienta? Możliwe opcje - cellular or telephone: ')
  if u_contact not in ['cellular', 'telephone']:
    print('Forma kontaktu nierozpoznana. Wprowadzono "unknown"')
    u_contact = 'unknown'
  return u_contact

def ask_month():
  while True:
    try:
      u_month = input('Podaj miesiąc ostatniego kontaktu z klientem. Możliwe opcje - jan, feb, mar, apr, jun, jul, aug, sep, oct, nov, dec: ')
    except u_month not in ['jan', 'feb', 'mar', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']: 
      print('Wybierz opcję z listy - jan, feb, mar, apr, jun, jul, aug, sep, oct, nov, dec')
      continue
    return u_month



def ask_day():
  while True:
     try:
         u_day = int(input("Wprowadź dzień miesiąca ostatniego kontaktu z klientem: "))
     except ValueError:
         print("Wprowadź liczbę - przykładowo 24")
         continue
     else:
         break
  if u_day >= 32 or u_day < 1: 
     print("Wprowadź poprawną wartość.  ")
     ask_day()
  else:
     return u_day

def ask_duration():
  while True:
     try:
         u_duration = float(input("Wprowadź czas trwania ostatniej rozmowy w sekundach: "))
     except ValueError:
         print("Wprowadź liczbę")
         continue
     else:
         return u_duration



def ask_campaign():
  u_campaign = int(input('Ile razy kontaktowano się z klientem podczas bieżącej kampanii: '))
  if u_campaign < 0:
    u_campaign = int(input('Wprowadzono niepoprawną wartość. Wprowadź jeszcze raz: '))
  return u_campaign


def ask_previous():
  u_previous = int(input('Ile razy kontaktowano się z klientem przed rozpoczęciem bieżącej kampanii?: '))
  if u_previous < 0:
    u_previous = int(input('Wprowadzono niepoprawną wartość. Wprowadź jeszcze raz: '))
  return u_previous

def ask_poutcome():
  u_poutcome = input('Wynik poprzedniej kampanii. Możliwe opcje - success, failure, other lub unknown: ')
  if u_poutcome not in ['success', 'failure', 'other', 'unknown']:
    print('Nierozpoznano')
    ask_poutcome()
  else:
    return u_poutcome



In [ ]:
def ask_for_input():
  global user_input
  user_input = []
  #ask_age()
  #ask_job()
  user_input = [ask_age(), ask_job(), ask_marital(), ask_education() ,ask_default(), ask_balance(), ask_credit(1), ask_credit(2), ask_contact()	, ask_day()	, ask_month()	, ask_duration()	, ask_campaign()	,	ask_previous(),	ask_poutcome()]
  confirmation()

#user_input = ask_for_input()

#inputdf = pd.DataFrame([user_input], columns=['age','job','marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'previous', 'poutcome'])
####nastepne kroki
#standaryzacja probki od uzytkonika tak jak probki uczacej i testowej
#predykcja modelem
#wystawienie wyniku


Wprowadzone przez użytkownika dane musimy przekształcić do formy, którą model będzie w stanie przyjąć za dane wejściowe, więc musi być przedstawiony w postaci jednego wiersza macierzy X. Poniższa funkcja, po potwierdzeniu użytkownika, przygotowuje danę przez skalowanie

In [ ]:
def confirmation():
  global inputdf, user_input_all_col, emptydf
  print("--------------------------------------------------------------------------------------")
  i = 0
  c = ['Wiek','Zawór','Stan cywilny', 'Wykształcenie', 'Status default', 'Stan konta', 'Kredyt hipoteczny', 'Kredyt gotówkowy', 'Forma kontaktu', 'Ostatni kontakt z klientem - dzień miesiąca', 'Ostatni kontakt z klientem - miesiąc', 'Czas trwania ostatniego kontaktu z klientem', 'Liczba kontaktów z klientem w bieżącej kampanii', 'Liczba kontaktów z klientem w poprzedniej kampanii', 'Wynik poprzedniej kampani']
  while i < 15:
    print(c[i] + " : " + str(user_input[i]))
    i = i + 1
  conf = input("Czy potwierdzasz dane powyżej? [Y/N]: ")
  if conf.upper() == 'Y':
    inputdf = pd.DataFrame([user_input], columns=['age','job','marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'previous', 'poutcome'])
    inputdf = pd.get_dummies(inputdf)
    emptydf = pd.DataFrame(columns = ['age',
    'balance',
    'day',
    'duration',
    'campaign',
    'previous',
    'job_admin.',
    'job_blue-collar',
    'job_entrepreneur',
    'job_housemaid',
    'job_management',
    'job_retired',
    'job_self-employed',
    'job_services',
    'job_student',
    'job_technician',
    'job_unemployed',
    'job_unknown',
    'marital_divorced',
    'marital_married',
    'marital_single',
    'education_primary',
    'education_secondary',
    'education_tertiary',
    'education_unknown',
    'default_no',
    'default_yes',
    'housing_no',
    'housing_yes',
    'loan_no',
    'loan_yes',
    'contact_cellular',
    'contact_telephone',
    'contact_unknown',
    'month_apr',
    'month_aug',
    'month_dec',
    'month_feb',
    'month_jan',
    'month_jul',
    'month_jun',
    'month_mar',
    'month_may',
    'month_nov',
    'month_oct',
    'month_sep',
    'poutcome_failure',
    'poutcome_other',
    'poutcome_success',
    'poutcome_unknown'])
    
    user_input_all_col = pd.merge(left = emptydf, right = inputdf, how = 'outer')
    user_input_all_col = user_input_all_col.fillna(0)
    #return(user_input_all_col)
    user_input_all_col = scaler.transform(user_input_all_col)
    #user_input_all_col = user_input_all_col[X_temp.columns]

    print("Wczytano dane")
  else:
    print("--------------------------------------------------------------------------------------")
    print("Wprowadź dane jeszcze raz")
    ask_for_input()


Dodatkowo, poniżej znajdują się funkcje do prognozy wyniku na podstawie modelu oraz funkcja for_user, która wywołuje poszczególne funkcje.




In [ ]:
def forecast():
  q = model.predict(user_input_all_col)[0,0]
  if q == 1: return 'Klient założy lokatę!'
  elif q == 0: return 'Klient nie założy lokaty'

def for_user():
  x = login_proc()
  if x == "Zalogowano!" :
    print("Zalogowano! \n -------------------------------------------------------------")
    ask_for_input()
    print("-------------------------------------------------- \n", forecast())
  elif x == 'Błędna nazwa użytkownika':
    print("Błędny email")
  elif x == "Nieprawidłowe hasło":
    print("Błędne hasło")


# Sekcja dla użytkownika
Wywołując poniższą komórkę, uruchamia się interfejs w którym użytkownik może sprawdzić czego może spodziewać się po kliencie, do którego ma zamiar zadzwonić.

In [ ]:
for_user()

Podaj adres email: admin@bank.pl
Podaj hasło: pass
Zalogowano! 
 -------------------------------------------------------------
Wprowadź wiek klienta: 12
Klient musi być pełnoletni.
Wprowadź wiek klienta: 30
Podaj zawód klienta (przykładowo: admin., management,student, retired): management
Wprowadź stan cywilny klienta. Możliwe opcje - married, single lub divorced: married
Wprowadź informację na temat wykształcenia klienta. Możliwe opcje - primary, secondary, tertiary: primary
Czy klient ma status default? Y/N: n
Wprowadź stan konta klienta: 800
Czy klient ma kredyt hipoteczny? Y/N: n
Czy klient ma kredyt konsumencki? Y/N: n
Jaka jest preferowana forma kontaktu klienta? Możliwe opcje - cellular or telephone: cellular
Wprowadź dzień miesiąca ostatniego kontaktu z klientem: 3
Podaj miesiąc ostatniego kontaktu z klientem. Możliwe opcje - jan, feb, mar, apr, jun, jul, aug, sep, oct, nov, dec: feb
Wprowadź czas trwania ostatniej rozmowy w sekundach: 790
Ile razy kontaktowano się z klientem p

Podgląd historii logowania

In [ ]:
login_history

,email,password_correct,email_correct,datetime
0,admin@b.pl,None,No,2021-02-10
1,admin@bank.pl,Yes,Yes,2021-02-10
